<a href="https://colab.research.google.com/github/kkairu/Keras_food_MNIST/blob/master/Keras_food_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# food MNIST


<img src=https://raw.githubusercontent.com/srohit0/food_mnist/master/images/food-collage.jpg  width="800" height="400" align="center">

In [0]:
%%time

#! rm -fr food_mnist
! git clone https://github.com/srohit0/food_mnist.git

import food_mnist
from google.colab.patches import cv2_imshow

fatal: destination path 'food_mnist' already exists and is not an empty directory.
CPU times: user 39.3 ms, sys: 19.3 ms, total: 58.7 ms
Wall time: 847 ms


## 1. Import Packages

In [0]:
import os
import numpy as np
import tensorflow as tf

import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten,Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from keras import regularizers

np.random.seed(0) 
tf.set_random_seed(2)

Using TensorFlow backend.


## 2. Load MNIST Dataset

In [0]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels



def divide_dataset(dataset, labels):
    train_pct = 0.80; 
    # divide dataset into training and validation set
    train_index = int(dataset.shape[0]*train_pct)
    t_X = dataset[:train_index, :]
    t_Y = labels[:train_index]
    v_X = dataset[train_index:,:]
    v_Y = labels[train_index:]
    
    return (t_X, t_Y), (v_X, v_Y)


In [0]:
batch_size = 1000
num_classes = 10
epochs = 100

# input image dimensions
img_rows, img_cols, img_channels = 56, 56, 3
input_shape = (img_rows, img_cols, img_channels)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = food_mnist.load_data(img_cols, img_rows)
X = np.concatenate((x_train, x_test), axis=0)
Y = np.concatenate((y_train, y_test), axis=0)

X, Y = randomize(X, Y)
(x_train, y_train), (x_test, y_test) = divide_dataset(X, Y)

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, img_channels)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, img_channels)

x_train = x_train.astype('float16') / 255.0
x_test = x_test.astype('float16') / 255.0


# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [0]:
def train_input_fn(batch_size=1000):
  # convert the inputs to a Dataset.
  dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))

  # shuffle, repeat, and batch the examples.
  dataset = dataset.shuffle(1000).repeat().batch(batch_size, drop_remainder=True)

  # return the dataset.
  return dataset

## 3. Define Deep Learning  Model

In [0]:
learning_rate = 0.001
filter_sz = 3
n_filters = 28

def create_model():
  Inp = tf.keras.Input(name='input', shape=input_shape, batch_size=None, dtype=tf.float32)

  x = Conv2D(n_filters, kernel_size=(filter_sz, filter_sz), activation='relu', kernel_regularizer=regularizers.l2(l=0.0001), name = 'Conv_01')(Inp)
  x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_01')(x)
  #  kernel_regularizer=regularizers.l2(l=0.00005)
  x = Conv2D(2*n_filters, (filter_sz, filter_sz), activation='relu', name = 'Conv_02')(x)
  x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_02')(x)
  x = Conv2D(2*n_filters, (filter_sz, filter_sz), activation='relu',  name = 'Conv_03')(x)
  x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_03')(x)
  x = Conv2D(n_filters, (filter_sz, filter_sz), activation='relu',  name = 'Conv_04')(x)
  x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_04')(x)
  x = Flatten(name = 'Flatten_01')(x)
  x = Dense(2*n_filters, activation='relu',name = 'Dense_01')(x)
  x = Dense(num_classes, activation='relu',name = 'Dense_02')(x)
  x = Dropout(0.5,name = 'Dropout_02')(x)
  output = Dense(num_classes, activation='softmax',name = 'Dense_out')(x)
  
  return tf.keras.models.Model(inputs=[Inp], outputs=[output])

In [0]:
tpu_grpc_url = "grpc://"+os.environ["COLAB_TPU_ADDR"]
cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_grpc_url)
tf.config.experimental_connect_to_host(cluster_resolver.master())
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
tpu_strategy = tf.distribute.experimental.TPUStrategy(cluster_resolver)

## 4. Train The Model

In [0]:
with tpu_strategy.scope():
  model = create_model()
  opt = tf.train.AdamOptimizer(learning_rate)
  #opt = tf.train.AdadeltaOptimizer()
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
  model.summary()
  model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
  score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
  print('\n########################################\n#   T E S T    A C C U R A C Y:', score[1]*100, "%\n########################################")

W0621 14:46:20.968342 140397540263808 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 56, 56, 3)]       0         
_________________________________________________________________
Conv_01 (Conv2D)             (None, 54, 54, 28)        784       
_________________________________________________________________
MaxPool_01 (MaxPooling2D)    (None, 27, 27, 28)        0         
_________________________________________________________________
Conv_02 (Conv2D)             (None, 25, 25, 56)        14168     
_________________________________________________________________
MaxPool_02 (MaxPooling2D)    (None, 12, 12, 56)        0         
_________________________________________________________________
Conv_03 (Conv2D)             (None, 10, 10, 56)        28280     
_________________________________________________________________
MaxPool_03 (MaxPooling2D)    (None, 5, 5, 56)          0     

W0621 14:46:25.319872 140397540263808 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/util/random_seed.py:58: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0621 14:46:30.871763 140397540263808 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training_distributed.py:411: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Epoch 1/100
4/4 [==============================] - 6s 1s/step - loss: 2.3030 - acc: 0.1028 - val_loss: 2.3024 - val_acc: 0.1240
Epoch 2/100
4/4 [==============================] - 5s 1s/step - loss: 2.3024 - acc: 0.1065 - val_loss: 2.3008 - val_acc: 0.1280
Epoch 3/100
4/4 [==============================] - 6s 2s/step - loss: 2.3004 - acc: 0.1258 - val_loss: 2.2990 - val_acc: 0.1410
Epoch 4/100
4/4 [==============================] - 7s 2s/step - loss: 2.2981 - acc: 0.1268 - val_loss: 2.2951 - val_acc: 0.1590
Epoch 5/100
4/4 [==============================] - 7s 2s/step - loss: 2.2946 - acc: 0.1268 - val_loss: 2.2895 - val_acc: 0.1410
Epoch 6/100
4/4 [==============================] - 8s 2s/step - loss: 2.2907 - acc: 0.1203 - val_loss: 2.2855 - val_acc: 0.1570
Epoch 7/100
4/4 [==============================] - 9s 2s/step - loss: 2.2890 - acc: 0.1348 - val_loss: 2.2800 - val_acc: 0.1580
Epoch 8/100
4/4 [==============================] - 9s 2s/step - loss: 2.2802 - acc: 0.1358 - val_loss: 2

## 5. Display !

In [0]:
from matplotlib import pyplot as plt
import random

labels_dict = food_mnist.labels()
img_class = model.predict(x_test, batch_size=batch_size)

for x in range(5):
  idx = random.randint(0,x_test.shape[0])
  img = x_test[idx]
  fig=plt.figure(figsize=(3,3), dpi=72, facecolor='w', edgecolor='k')
  plt.axis('off')
  plt.imshow(img)
  plt.title("food test " + str(idx) + ": " + labels_dict[np.argmax(img_class[idx])] + 
            " (" + labels_dict[np.argmax(y_test[idx])] + ")" )
  
  plt.show()